In [1]:
import cv2
import numpy as np
import pytesseract
import os
from matplotlib import pyplot as plt 

In [2]:
per = 25
pytesseract.pytesseract.tesseract_cmd =  'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [3]:
imgQ = cv2.imread('img.jpg')
h,w,c = imgQ.shape
# imgQ = cv2.resize(imgQ,(w//1,h//1))


In [4]:
#       hl    vu    hr   vd
# roi = [[(4, 380), (525, 477), 'Text', 'table'],
#     [(306, 33), (402, 130), 'Text', 'Date'], 
#        [(413, 56), (572, 70), 'Text', 'Invoice Id'], 
#        [(26, 126), (172, 140), 'Text', 'Buyer Name'], 
#        [(26, 142), (197, 169), 'Text', 'Buyer Address'], 
#        [(26, 170), (168, 186), 'Text', 'Buyer Number'], 
#        [(26, 186), (198, 215), 'Text', 'Buyer Mail'],
#        [(413, 32), (520, 52), 'Text', 'Date'],
#        [(413, 73), (508, 98), 'Text', 'PO Number']]

# roi = Region of Interest
roi = [[(410, 31), (578, 102), 'Text', 'Date Details'],
       [(15, 114), (220, 226), 'Text', 'Buyer Details'],
       [(3, 374), (537, 485), 'Text', 'Table Details'], 
       [(157, 500), (399, 607), 'Text', 'Other Details']]

        
# roi = [[(53, 1444), (880, 1486), 'text', 'addres']]
# hl - horizontal left , vu = vertical up , hr , horizontal right , vd = vertical down 
# hl-0 , vu -0 , hr - 592 , vd -830 

In [5]:
# cv2.imshow('output',imgQ)
# cv2.waitKey(0)

In [6]:
orb = cv2.ORB_create(20000)
kp1 , des1 = orb.detectAndCompute(imgQ, None)
# imgkp1 = cv2.drawKeypoints(imgQ,kp1,None)

In [7]:
# cv2.imshow("imagekp1",imgkp1)
# cv2.imshow('output',imgQ)
# cv2.waitKey(0)

In [9]:
path = 'C:\\Users\\virtu\\Downloads\\Data\\Tesseract_OCR'
myPicList = os.listdir(path)
# print(myPicList)

for j,y in enumerate(myPicList):
    img = cv2.imread(path + "/" + y)
    # img = cv2.resize(img,(w//2,h//2))
    # cv2.imshow(y,img)
    # cv2.waitKey(0)
    kp2,des2 = orb.detectAndCompute(img, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.match(des2,des1)
    # matches.sort(key=lambda x:x.distance)
    good = matches[: int(len(matches)*(per/100))]
    imgMatch = cv2.drawMatches(img,kp2,imgQ,kp1,good[:100],None , flags=2)
    # imgMatch = cv2.resize(imgMatch,(w//3,h//3))
    # cv2.imshow(y,imgMatch)
    # cv2.waitKey(0)

    srcPoints = np.float32([kp2[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dstPoints = np.float32([kp1[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    M,_ = cv2.findHomography(srcPoints,dstPoints,cv2.RANSAC,5.0)
    imgScan = cv2.warpPerspective(img,M,(w,h))
    # imgScan = cv2.resize(imgScan,(w//3,h//3))
    # cv2.imshow(y,imgScan)
    # cv2.waitKey(0)
    imgShow = imgScan.copy()
    imgMask = np.zeros_like(imgShow)
    print(imgMask)

    myData = []

    print(f"######## Extracting the data from {y} #######")
    for x , r in enumerate(roi):
        cv2.rectangle(imgMask ,((r[0][0]),r[0][1]),((r[1][0]),r[1][1]) ,(0,255,0) ,cv2.FILLED )
        imgShow = cv2.addWeighted(imgShow ,0.99,imgMask,0.1,0)
        # cv2.imshow(y,imgShow)
        # cv2.waitKey(0)

        
        imgCrop = imgScan[ r[0][1]:r[1][1] , r[0][0]:r[1][0] ]
        # cv2.imshow(str(x),imgCrop)
        # cv2.waitKey(0)

        if r[2] == 'Text' :
            print(f' {r[3]} : {pytesseract.image_to_string(imgCrop)} ')
            myData.append(pytesseract.image_to_string(imgCrop))

    print()
        
    # imgShow = cv2.resize(imgShow,(w//3,h//3))
    # cv2.imshow(y,imgShow)
    # cv2.waitKey(0)

    
    
    

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
######## Extracting the data from Template5_Instance0.jpg #######
 Date Details : Date: 18-Jan-1999
INVOICE ID 5Y3M7d-590
PO Number

 
 Buyer Details : Bill to:Robyn Johnson
233 Foster Gardens
Johnburgh, 1D 41900 US

Site:http:J/www jones.biz/
 
 Table Details : ry

‘ay Description Unitprice | Amount
400 572372 | Speak cultural 5.86 EY
600 (060367 | Apply student card 587, 51522,
300 490537 | Economy people. 984 29.52
1.00 477174 | Human suggest ree 4927 4927

 
 Other Details : ‘SUB_TOTAL : 677.45 USD
TAXVAT (3.49%): 23.66 USD
BALANCE DUE : 677.33 USD

Total in words: six hundre

In [7]:
roi = [[(834, 82), (1044, 98), 'Text', 'Date'], 
       [(832, 118), (1122, 136), 'Text', 'Invoice Id'], 
       [(54, 256), (272, 276), 'Text ', 'Buyer Name'], 
       [(56, 288), (394, 334), 'Text', 'Buyer Address'], 
       [(52, 340), (276, 362), 'Text', 'Buyer Number'], 
       [(58, 372), (378, 390), 'Text', 'Buyer Mail'],
       [(832, 146), (998, 170), 'Text', 'PO Number']]

In [8]:
imgShow = imgQ.copy()
imgMask = np.zeros_like(imgShow)

In [10]:
for x , r in enumerate(roi):

    cv2.rectangle(imgMask ,((r[0][0]),r[0][1]),((r[1][0]),r[1][1]) ,(0,255,0) ,cv2.FILLED )
    imgShow = cv2.addWeighted(imgShow ,0.99,imgMask,0.1,0)
imgQ = cv2.resize(imgQ,(w//0.5,h//0.5))   
cv2.imshow("2",imgShow)
cv2.waitKey(0)

-1